In [1]:
import json
import pandas as pd

In [3]:
from datasets import load_dataset

# 데이터셋 불러오기
raw_datasets = load_dataset('starmpcc/Asclepius-Synthetic-Clinical-Notes', 'train')
raw_datasets

# 판다스 데이터프레임으로 변환
dataset = pd.DataFrame(raw_datasets['train'].to_pandas())
dataset

# 추가 컬럼 생성
dataset['korean'] = ""
dataset['mixed'] = ""
dataset['response'] = ""

Found cached dataset csv (/Users/serimkim/.cache/huggingface/datasets/starmpcc___csv/starmpcc--Asclepius-Synthetic-Clinical-Notes-a11a728105d5eede/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
# prompt에 넘겨줄 예제 불러오기

notes = pd.read_excel('notes.xlsx')
samples = notes[:50]

In [5]:
# api 호출 함수 정의

from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="",
)
def gpt_response(input_text):
    prompt = make_prompt(input_text)

    # OpenAI API 호출
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a translator from English to a mixed text of English and Korean."},
            {"role": "user", "content": prompt}
        ]
    )
    answer = response.choices[0].message.content
    return answer

In [6]:
# prompt로 넘겨줄 예제 랜덤 샘플링하는 함수 정의

import random

def make_prompt(input_text):
    # 0부터 49까지의 숫자 리스트 생성
    numbers = list(range(50))

    # 리스트에서 3개의 랜덤한 숫자 선택
    random_numbers = random.sample(numbers, 3)

    prompt = """For the given ‘english’ text, generate ‘korean’ and ‘mixed’ in json format.
‘Korean’ is a translation of ‘english’ into Korean, and ‘mixed’ is a translation of English and Korean mixed together.
<sample1> ~ <sample3> is a example of creating ‘korean’ and ‘mixed’ from ‘english’.
"""

    for i in [1,2,3]:
        prompt += f"\n\n<sample{i}>\nenglish: {samples['snippet'][i]}\n"
        prompt += f"{{\"korean\":\"{samples['korean'][i]}\", \"mixed\": \"{samples['mixed'][i]}\"}}"

    prompt += "\n\nAs in the examples above, write ‘korean’ and ‘mixed’ for ‘English’ below. please just return json.\n\n"

    prompt += input_text

    return prompt

In [9]:
import time

start_time = time.time()
for i in range(250, 400):
    response = gpt_response(dataset['note'][i].replace('\n', ''))
    print(i)

    try:
        response = json.loads(response.replace('\n', ''))
        dataset['korean'][i] = response.get('korean', 'Error: Korean data not found')
        dataset['mixed'][i] = response.get('mixed', 'Error: Mixed data not found')
        if i>250 and i % 50 == 0:
            elapsed_time = time.time() - start_time
            print(f"Elapsed Time for {i} iterations: {elapsed_time} seconds")
            num = i
            gpt_data = dataset[i-50:i]
            gpt_data.to_excel(f'gpt_data_{i}.xlsx')
            start_time = elapsed_time
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON for index {i}: {e}")
        # 에러 발생 시 스킵하고 다음 i에 대해 반복
        continue
    except KeyError as e:
        print(f"Error accessing key in response for index {i}: {e}")
        dataset['response'][i] = response
        # 에러 발생 시 스킵하고 다음 i에 대해 반복
        continue

print('===================== Augmentation is done. ====================')
gpt_data = dataset[350:400]
gpt_data.to_excel(f'gpt_data_400.xlsx')

450


/var/folders/16/llg4mhsx5yvgfmnhmknw_l8w0000gn/T/ipykernel_9343/4014350804.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['korean'][i] = response.get('korean', 'Error: Korean data not found')
/var/folders/16/llg4mhsx5yvgfmnhmknw_l8w0000gn/T/ipykernel_9343/4014350804.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['mixed'][i] = response.get('mixed', 'Error: Mixed data not found')


451


/var/folders/16/llg4mhsx5yvgfmnhmknw_l8w0000gn/T/ipykernel_9343/4014350804.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['korean'][i] = response.get('korean', 'Error: Korean data not found')
/var/folders/16/llg4mhsx5yvgfmnhmknw_l8w0000gn/T/ipykernel_9343/4014350804.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['mixed'][i] = response.get('mixed', 'Error: Mixed data not found')


452


...

/var/folders/16/llg4mhsx5yvgfmnhmknw_l8w0000gn/T/ipykernel_9343/4014350804.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['korean'][i] = response.get('korean', 'Error: Korean data not found')
/var/folders/16/llg4mhsx5yvgfmnhmknw_l8w0000gn/T/ipykernel_9343/4014350804.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['mixed'][i] = response.get('mixed', 'Error: Mixed data not found')


498


/var/folders/16/llg4mhsx5yvgfmnhmknw_l8w0000gn/T/ipykernel_9343/4014350804.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['korean'][i] = response.get('korean', 'Error: Korean data not found')
/var/folders/16/llg4mhsx5yvgfmnhmknw_l8w0000gn/T/ipykernel_9343/4014350804.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['mixed'][i] = response.get('mixed', 'Error: Mixed data not found')


499
===================== Augmentation is done. ====================


/var/folders/16/llg4mhsx5yvgfmnhmknw_l8w0000gn/T/ipykernel_9343/4014350804.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['korean'][i] = response.get('korean', 'Error: Korean data not found')
/var/folders/16/llg4mhsx5yvgfmnhmknw_l8w0000gn/T/ipykernel_9343/4014350804.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['mixed'][i] = response.get('mixed', 'Error: Mixed data not found')


In [8]:
gpt_data = dataset[200:250]
gpt_data.to_excel(f'gpt_data_250.xlsx')

## data concat

In [2]:
import os

# 현재 디렉토리에서 파일 목록 가져오기
file_list = os.listdir()

# ".xlsx"로 끝나는 파일들만 골라내기
xlsx_files = [file for file in file_list if file.startswith("gpt_data") and file.endswith(".xlsx")]

In [8]:
xlsx_files = ['gpt_data_200.xlsx',
 'gpt_data_450.xlsx',
 'gpt_data_400.xlsx',
 'gpt_data_250.xlsx',
 'gpt_data_300.xlsx',
 'gpt_data_100.xlsx',
 'gpt_data_500.xlsx',
 'gpt_data_150.xlsx',
 'gpt_data_350.xlsx']

In [9]:
df = pd.read_excel('gpt_data_50.xlsx')

for file in xlsx_files:
    # 행 방향으로 합치기 (아래로 연결)
    df_sub = pd.read_excel(file)
    df = pd.concat([df, df_sub], ignore_index=True)

In [10]:
len(df)

500

In [11]:
df.to_excel('gpt_data_.xlsx')